In [2]:
import requests
import pandas as pd
import pymysql
import numpy as np
from datetime import datetime

In [3]:
mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="12212112",
    database="gitdata"
)

mycursor = mydb.cursor()

In [4]:
# Topics to fetch data for
topics = ["Gen AI","Powerbi", "deep learning", "MLOps",
          "Quantum Computing", "python", "machine learning","robotics"]

In [5]:
headers = {'Authorization': 'YOUR_TOKEN'}

In [6]:
def fetch_github_data(topic):
    url = f"https://api.github.com/search/repositories?q={topic}&sort=stars"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for {topic}")
        return None

# Extract data for all topics
data_list = []
for topic in topics:
    data = fetch_github_data(topic)
    if data:
        for item in data['items']:
            data_list.append({
                'Repository_Name': item['name'],
                'Owner': item['owner']['login'],
                'Description': item['description'],
                'URL': item['html_url'],
                'Programming_Language': item.get('language', 'Unknown'),
                'Creation_Date': item['created_at'],
                'Last_Updated_Date': item['updated_at'],
                'Number_of_Stars': item['stargazers_count'],
                'Number_of_Forks': item['forks_count'],
                'Number_of_Open_Issues': item['open_issues_count'],
                'License_Type': item['license']['name'] if item['license'] else 'No License'
            })

df = pd.DataFrame(data_list)
df.to_csv('github_repositor.csv', index=False)
print(df)

Failed to fetch data for machine learning
Failed to fetch data for robotics
                 Repository_Name                Owner  \
0    generative-ai-for-beginners            microsoft   
1              generative-models         Stability-AI   
2                        danswer           danswer-ai   
3    awesome-generative-ai-guide          aishwaryanr   
4                  generative-ai  GoogleCloudPlatform   
..                           ...                  ...   
175                      cpython               python   
176                       d2l-zh               d2l-ai   
177                      ansible              ansible   
178                     superset               apache   
179                        keras           keras-team   

                                           Description  \
0    21 Lessons, Get Started Building with Generati...   
1                    Generative Models by Stability AI   
2    Gen-AI Chat for Teams - Think ChatGPT if it ha...   
3    A 

In [7]:
# Data Cleaning
print(df.shape)
print(df.info()) # show full info
print(df.isna().sum()) # check null

(180, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Repository_Name        180 non-null    object
 1   Owner                  180 non-null    object
 2   Description            175 non-null    object
 3   URL                    180 non-null    object
 4   Programming_Language   154 non-null    object
 5   Creation_Date          180 non-null    object
 6   Last_Updated_Date      180 non-null    object
 7   Number_of_Stars        180 non-null    int64 
 8   Number_of_Forks        180 non-null    int64 
 9   Number_of_Open_Issues  180 non-null    int64 
 10  License_Type           180 non-null    object
dtypes: int64(3), object(8)
memory usage: 15.6+ KB
None
Repository_Name           0
Owner                     0
Description               5
URL                       0
Programming_Language     26
Creation_Date             

In [9]:
df['Creation_Date'] = pd.to_datetime(df['Creation_Date']) # convert data type
df['Last_Updated_Date'] = pd.to_datetime(df['Last_Updated_Date'])

# HANDLING NULL VALUES

df = df.replace({np.nan: None})
df.fillna({'Description': 'No Description', 'License_Type': 'No License'}, inplace=True)

# Preview cleaned data
print(df.head())

               Repository_Name                Owner  \
0  generative-ai-for-beginners            microsoft   
1            generative-models         Stability-AI   
2                      danswer           danswer-ai   
3  awesome-generative-ai-guide          aishwaryanr   
4                generative-ai  GoogleCloudPlatform   

                                         Description  \
0  21 Lessons, Get Started Building with Generati...   
1                  Generative Models by Stability AI   
2  Gen-AI Chat for Teams - Think ChatGPT if it ha...   
3  A one stop repository for generative AI resear...   
4  Sample code and notebooks for Generative AI on...   

                                                 URL Programming_Language  \
0  https://github.com/microsoft/generative-ai-for...     Jupyter Notebook   
1  https://github.com/Stability-AI/generative-models               Python   
2              https://github.com/danswer-ai/danswer               Python   
3  https://github.com/ai

In [10]:
import pandas as pd
import numpy as np

# Load CSV 
df = pd.read_csv("github_repositor.csv")

# Convert NaN to None 
df = df.replace({np.nan: None})

df['Creation_Date'] = pd.to_datetime(df['Creation_Date']).dt.date
df['Last_Updated_Date'] = pd.to_datetime(df['Last_Updated_Date']).dt.date



# Create the repositories table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS reposit (
    Repository_Name VARCHAR(255) NOT NULL,
    Owner VARCHAR(255) NOT NULL,
    Description TEXT,
    URL VARCHAR(255) NOT NULL,
    Programming_Language VARCHAR(50),
    Creation_Date DATE,
    Last_Updated_Date DATE,
    Number_of_Stars INT,
    Number_of_Forks INT,
    Number_of_Open_Issues INT,
    License_Type VARCHAR(100)
);
"""
mycursor.execute(create_table_query)

# Insert DataFrame into MySQL table
insert_query = """
    INSERT INTO reposit (
        Repository_Name, 
        Owner, 
        Description, 
        URL, 
        Programming_Language, 
        Creation_Date, 
        Last_Updated_Date, 
        Number_of_Stars, 
        Number_of_Forks, 
        Number_of_Open_Issues, 
        License_Type
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for index, row in df.iterrows():
    val = (
        row['Repository_Name'], 
        row['Owner'], 
        row['Description'], 
        row['URL'], 
        row['Programming_Language'], 
        row['Creation_Date'], 
        row['Last_Updated_Date'], 
        row['Number_of_Stars'], 
        row['Number_of_Forks'], 
        row['Number_of_Open_Issues'], 
        row['License_Type']
    )
    
    try:
        mycursor.execute(insert_query, val)
    except pymysql.Error as err:
        print(f"Error: {err}")

# Commit the transaction
mydb.commit()
print("Data saved to MySQL database!")

Data saved to MySQL database!


In [11]:
# Example Analysis

# Top 5 prog lang by the number of repositories
top_languages = df['Programming_Language'].value_counts().head(5)
print("Top 5 Programming Languages:")
print(top_languages)

# Top 5 repositories with the most stars
top_starred_repos = df[['Repository_Name', 'Number_of_Stars']].sort_values(by='Number_of_Stars', ascending=False).head(5)
print("\nTop 5 Starred Repositories:")
print(top_starred_repos)

# Average number of stars by programming language
avg_stars_per_language = df.groupby('Programming_Language')['Number_of_Stars'].mean().sort_values(ascending=False)
print("\nAverage Stars per Language:")
print(avg_stars_per_language.head(5))

Top 5 Programming Languages:
Programming_Language
Python              73
Jupyter Notebook    24
JavaScript           9
C++                  9
TypeScript           7
Name: count, dtype: int64

Top 5 Starred Repositories:
            Repository_Name  Number_of_Stars
150    system-design-primer           272608
151          awesome-python           221652
152  project-based-learning           200870
153                  Python           191481
154              tensorflow           186044

Average Stars per Language:
Programming_Language
Shell     86769.500000
Java      48530.000000
Python    40578.082192
HTML      31944.666667
C++       31417.888889
Name: Number_of_Stars, dtype: float64
